After the last set of regressions, we determined that it makes more sense to develop models based on the entire **set** of sequences, rather than each sequence individually. To that end, the following regressions are aggregate models, using cross-validation and grid search as needed. As usual, the first step is Data Wrangling.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
%matplotlib inline

In [2]:
df = pd.read_csv('train.csv')
df.head()

,Id,Sequence
0,3,"1,3,13,87,1053,28576,2141733,508147108,4021352..."
1,7,"1,2,1,5,5,1,11,16,7,1,23,44,30,9,1,47,112,104,..."
2,8,"1,2,4,5,8,10,16,20,32,40,64,80,128,160,256,320..."
3,11,"1,8,25,83,274,2275,132224,1060067,3312425,1099..."
4,13,"1,111,12211,1343211,147753211,16252853211,1787..."


In [3]:
df.Sequence = df.Sequence.str.split(",")
df.head()

,Id,Sequence
0,3,"[1, 3, 13, 87, 1053, 28576, 2141733, 508147108..."
1,7,"[1, 2, 1, 5, 5, 1, 11, 16, 7, 1, 23, 44, 30, 9..."
2,8,"[1, 2, 4, 5, 8, 10, 16, 20, 32, 40, 64, 80, 12..."
3,11,"[1, 8, 25, 83, 274, 2275, 132224, 1060067, 331..."
4,13,"[1, 111, 12211, 1343211, 147753211, 1625285321..."


In [4]:
def conv(a):
    return list(map(int, a))
df['Sequence'] = df.Sequence.apply(conv)
df.head()

,Id,Sequence
0,3,"[1, 3, 13, 87, 1053, 28576, 2141733, 508147108..."
1,7,"[1, 2, 1, 5, 5, 1, 11, 16, 7, 1, 23, 44, 30, 9..."
2,8,"[1, 2, 4, 5, 8, 10, 16, 20, 32, 40, 64, 80, 12..."
3,11,"[1, 8, 25, 83, 274, 2275, 132224, 1060067, 331..."
4,13,"[1, 111, 12211, 1343211, 147753211, 1625285321..."


One of the big variables between sequences has been the length. To develop accurate models, it makes sense to standardize this aspect. Thus, the sequences will be subdivided into smaller, 10-number sequences.

In [5]:
def subdivide(a):
    return [a[i*10: (i+1)*10] for i in range(len(a) // 10)]

df['Subdivided'] = df.Sequence.apply(subdivide)
df.head()

,Id,Sequence,Subdivided
0,3,"[1, 3, 13, 87, 1053, 28576, 2141733, 508147108...","[[1, 3, 13, 87, 1053, 28576, 2141733, 50814710..."
1,7,"[1, 2, 1, 5, 5, 1, 11, 16, 7, 1, 23, 44, 30, 9...","[[1, 2, 1, 5, 5, 1, 11, 16, 7, 1], [23, 44, 30..."
2,8,"[1, 2, 4, 5, 8, 10, 16, 20, 32, 40, 64, 80, 12...","[[1, 2, 4, 5, 8, 10, 16, 20, 32, 40], [64, 80,..."
3,11,"[1, 8, 25, 83, 274, 2275, 132224, 1060067, 331...","[[1, 8, 25, 83, 274, 2275, 132224, 1060067, 33..."
4,13,"[1, 111, 12211, 1343211, 147753211, 1625285321...","[[1, 111, 12211, 1343211, 147753211, 162528532..."


In [6]:
seqs = []
for i in range(len(df)):
    for j in range(len(df.Subdivided[i])):
        seqs.append(df.Subdivided[i][j])

In [7]:
seqs = pd.Series(seqs)
seqs.head()

0    [1, 3, 13, 87, 1053, 28576, 2141733, 508147108...
1                     [1, 2, 1, 5, 5, 1, 11, 16, 7, 1]
2             [23, 44, 30, 9, 1, 47, 112, 104, 48, 11]
3          [1, 95, 272, 320, 200, 70, 13, 1, 191, 640]
4    [912, 720, 340, 96, 15, 1, 383, 1472, 2464, 2352]
dtype: object

In [8]:
seqs = pd.DataFrame(seqs, columns = ['Sequence'])
seqs.head()

,Sequence
0,"[1, 3, 13, 87, 1053, 28576, 2141733, 508147108..."
1,"[1, 2, 1, 5, 5, 1, 11, 16, 7, 1]"
2,"[23, 44, 30, 9, 1, 47, 112, 104, 48, 11]"
3,"[1, 95, 272, 320, 200, 70, 13, 1, 191, 640]"
4,"[912, 720, 340, 96, 15, 1, 383, 1472, 2464, 2352]"


In [9]:
def last(a):
    return a[-1]
seqs['Next'] = seqs.Sequence.apply(last)
seqs.head()

,Sequence,Next
0,"[1, 3, 13, 87, 1053, 28576, 2141733, 508147108...",1073376057490373
1,"[1, 2, 1, 5, 5, 1, 11, 16, 7, 1]",1
2,"[23, 44, 30, 9, 1, 47, 112, 104, 48, 11]",11
3,"[1, 95, 272, 320, 200, 70, 13, 1, 191, 640]",640
4,"[912, 720, 340, 96, 15, 1, 383, 1472, 2464, 2352]",2352


In [10]:
def trim(a):
    return a[:-1]
seqs['Sequence'] = seqs.Sequence.apply(trim)
seqs.head()

,Sequence,Next
0,"[1, 3, 13, 87, 1053, 28576, 2141733, 508147108...",1073376057490373
1,"[1, 2, 1, 5, 5, 1, 11, 16, 7]",1
2,"[23, 44, 30, 9, 1, 47, 112, 104, 48]",11
3,"[1, 95, 272, 320, 200, 70, 13, 1, 191]",640
4,"[912, 720, 340, 96, 15, 1, 383, 1472, 2464]",2352


To aid in the regressions, we will create features. Features help a classifier or regressor mathematically come up with more accurate models. Natural choices include basic summary statistics such as the mean and standard deviation.

In [11]:
def mean(a):
    return sum(a)/len(a)
seqs['Mean'] = seqs.Sequence.apply(mean)

def std(a):
    return np.std(a)
seqs['Standard Deviation'] = seqs.Sequence.apply(std)

seqs.head()

,Sequence,Next,Mean,Standard Deviation
0,"[1, 3, 13, 87, 1053, 28576, 2141733, 508147108...",1073376057490373,4.473840e+10,1.263590e+11
1,"[1, 2, 1, 5, 5, 1, 11, 16, 7]",1,5.444444e+00,4.901499e+00
2,"[23, 44, 30, 9, 1, 47, 112, 104, 48]",11,4.644444e+01,3.637188e+01
3,"[1, 95, 272, 320, 200, 70, 13, 1, 191]",640,1.292222e+02,1.138593e+02
4,"[912, 720, 340, 96, 15, 1, 383, 1472, 2464]",2352,7.114444e+02,7.677222e+02


Another good feature of a sequence is its **autocorrelation**, the correlation of a sequence with itself at different timelags. This gives an estimate of the predictive power of earlier elements on future ones. The aggregate autocorrelation is here defined as the average squared autocorrelation amongst all valid timelags. 

Note that the neglog function is again used to shrink the massive numbers while preserving sign.

In [12]:
import math

def neglog(a):
    b = a.copy()
    for i in range(len(a)):
        if a[i] < 0:
            b[i] = -1 * math.log(float(a[i] * -1))
        elif b[i] > 0:
            b[i] = math.log(float(a[i]))
        b[i] = float(b[i])
    return b

def agg_acf(a, length=8):
    x = neglog(a)
    b = np.array([1]+[np.corrcoef(x[:-i], x[i:])[0, 1] \
        for i in range(1, length)])
    b = np.nan_to_num(b)
    agg = 0
    for i in range(len(b)):
        agg += b[i] ** 2
    return agg/len(b)
    
seqs['Aggregate Autocorrelation'] = seqs.Sequence.apply(agg_acf)
seqs.head()

,Sequence,Next,Mean,Standard Deviation,Aggregate Autocorrelation
0,"[1, 3, 13, 87, 1053, 28576, 2141733, 508147108...",1073376057490373,4.473840e+10,1.263590e+11,0.999602
1,"[1, 2, 1, 5, 5, 1, 11, 16, 7]",1,5.444444e+00,4.901499e+00,0.423735
2,"[23, 44, 30, 9, 1, 47, 112, 104, 48]",11,4.644444e+01,3.637188e+01,0.394310
3,"[1, 95, 272, 320, 200, 70, 13, 1, 191]",640,1.292222e+02,1.138593e+02,0.358572
4,"[912, 720, 340, 96, 15, 1, 383, 1472, 2464]",2352,7.114444e+02,7.677222e+02,0.592345


Exploration of the dataframe shows that the sequences with the highest autocorrelation (those closest to 1) are almost entirely exponential. As the exponential function is it's own derivative, its growth rate maintains similar proportionality during different parts of the series and contributes to a higher autocorrelation/predictive power. 

The neglog function will be applied to the mean and standard deviation for consistency and ease of data manipulation.

In [13]:
def neglog2(a):
    if a < 0:
        return -1 * math.log(float(a * -1))
    if a == 0:
        return 0
    else:
        return math.log(float(a))

seqs['Mean'] = seqs.Mean.apply(neglog2)
seqs['Standard Deviation'] = seqs['Standard Deviation'].apply(neglog2)

seqs.rename(columns = {'Mean':'Log Mean', 'Standard Deviation': 'Log Dev'}, inplace = True)
seqs.head()

,Sequence,Next,Log Mean,Log Dev,Aggregate Autocorrelation
0,"[1, 3, 13, 87, 1053, 28576, 2141733, 508147108...",1073376057490373,24.524098,25.562393,0.999602
1,"[1, 2, 1, 5, 5, 1, 11, 16, 7]",1,1.694596,1.589541,0.423735
2,"[23, 44, 30, 9, 1, 47, 112, 104, 48]",11,3.838257,3.593796,0.394310
3,"[1, 95, 272, 320, 200, 70, 13, 1, 191]",640,4.861534,4.734964,0.358572
4,"[912, 720, 340, 96, 15, 1, 383, 1472, 2464]",2352,6.567297,6.643428,0.592345


The autocorrelation describes the predictive power of earlier elements of the sequence. Thus, it makes sense to use the two previous values in the sequence as features, so the regressor can make better predictions.

Note that the previous regressions relied entirely on the elements of each individual sequence. Here, we use a mixture of the actual terms, summary statistics, and descriptors of the sequences as a whole for prediction.

In [14]:
def slast(a):
    return a[-2]

seqs['Log SLast'] = seqs.Sequence.apply(slast).apply(neglog2)
seqs['Log Last'] = seqs.Sequence.apply(last).apply(neglog2)

seqs.head()

,Sequence,Next,Log Mean,Log Dev,Aggregate Autocorrelation,Log SLast,Log Last
0,"[1, 3, 13, 87, 1053, 28576, 2141733, 508147108...",1073376057490373,24.524098,25.562393,0.999602,20.046282,26.720054
1,"[1, 2, 1, 5, 5, 1, 11, 16, 7]",1,1.694596,1.589541,0.423735,2.772589,1.945910
2,"[23, 44, 30, 9, 1, 47, 112, 104, 48]",11,3.838257,3.593796,0.394310,4.644391,3.871201
3,"[1, 95, 272, 320, 200, 70, 13, 1, 191]",640,4.861534,4.734964,0.358572,0.000000,5.252273
4,"[912, 720, 340, 96, 15, 1, 383, 1472, 2464]",2352,6.567297,6.643428,0.592345,7.294377,7.809541


The next step is to finalize the dataframes for use in our predictive models. This involves splitting the entire frame into one of features and one with the target number.

In [15]:
X = seqs.drop('Next', axis = 1)
X.head()

,Sequence,Log Mean,Log Dev,Aggregate Autocorrelation,Log SLast,Log Last
0,"[1, 3, 13, 87, 1053, 28576, 2141733, 508147108...",24.524098,25.562393,0.999602,20.046282,26.720054
1,"[1, 2, 1, 5, 5, 1, 11, 16, 7]",1.694596,1.589541,0.423735,2.772589,1.945910
2,"[23, 44, 30, 9, 1, 47, 112, 104, 48]",3.838257,3.593796,0.394310,4.644391,3.871201
3,"[1, 95, 272, 320, 200, 70, 13, 1, 191]",4.861534,4.734964,0.358572,0.000000,5.252273
4,"[912, 720, 340, 96, 15, 1, 383, 1472, 2464]",6.567297,6.643428,0.592345,7.294377,7.809541


In [16]:
y = seqs.Next.apply(neglog2)
y.head()

0    34.609585
1     0.000000
2     2.397895
3     6.461468
4     7.763021
Name: Next, dtype: float64

In [17]:
ft = X.drop('Sequence', axis = 1)
ft.head()

,Log Mean,Log Dev,Aggregate Autocorrelation,Log SLast,Log Last
0,24.524098,25.562393,0.999602,20.046282,26.720054
1,1.694596,1.589541,0.423735,2.772589,1.945910
2,3.838257,3.593796,0.394310,4.644391,3.871201
3,4.861534,4.734964,0.358572,0.000000,5.252273
4,6.567297,6.643428,0.592345,7.294377,7.809541


First, we test basic linear regression - fit over the entire set.

In [18]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression()
lm.fit(ft, y)
y_lm = lm.predict(ft)

lm.score(ft, y)

0.81076744147652668

In [29]:
ft_train, ft_test, y_train, y_test = train_test_split(ft, y, test_size = .33, random_state=5)

cross_lm = LinearRegression()
cross_lm.fit(ft_train, y_train)
y_cross_lm = cross_lm.predict(ft_test)

cross_lm.score(ft_test, y_test)

0.80619678332781886

The linear regression performed substantially better than in the previous set! The use of the entire set and various features definitely made an impact.

A different type of regressor uses Support Vector Machines. SVMs are good when the data can't be modeled by just a simple line. It makes sense to try this as perhaps a linear model is a naive approach to the problem.

The first step is to grid search for the optimal C parameter.

In [19]:
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
from sklearn.svm import LinearSVR

# scoring function to test regressors
def cv_score(svr, X, y):
    total = len(X)
    n_folds = 4
    kf = KFold(total, n_folds=n_folds, shuffle=True)
    result = 0

    for train, test in kf:
        X_train = X.loc[train]
        X_test = X.loc[test]
        y_train = y[train]
        y_test = y[test]
        X_train = X_train[np.isfinite(X_train['Log Mean'])]
        X_test = X_test[np.isfinite(X_test['Log Mean'])]
        y_train = y_train[np.isfinite(y_train)]
        y_test = y_test[np.isfinite(y_test)]
        svr.fit(X_train, y_train)
        svr.predict(X_test)
        result += svr.score(X_test, y_test)
    return result / n_folds

#the grid of parameters to search over
Cs = [0.001, 0.1, 1, 10, 100]

ft_train, ft_test, y_train, y_test = train_test_split(ft, y, test_size = .33, random_state=5)

max = 0
maxc = 0
for i in Cs:
    clsvr = LinearSVR(C = i)
    score = cv_score(clsvr, ft_train, y_train)
    print(score)
    if score > max:
        max = score
        maxc = i
print("The C with the highest averages score is " + str(maxc))

0.795059366585
0.794847233978
0.793952715844
0.790617325888
0.448172992655
The C with the highest averages score is 0.001


The LinearSVR is used here because a kernelized SVR does not scale well. The LinearSVR sacrifices accuracy for speed, and with the massive data set we are working with, the speedup was needed for the model to finish training in adequate time. 

The model was first tested over the entire set, and then with a train-test split.

In [20]:
lsvr = LinearSVR(C = maxc)
lsvr.fit(ft, y)
y_lsvr = lsvr.predict(ft)

lsvr.score(ft, y)

0.7909078538498302

In [21]:
ft_train, ft_test, y_train, y_test = train_test_split(ft, y, test_size = .33, random_state=5)

cross_lsvr = LinearSVR(C = maxc)
cross_lsvr.fit(ft_train, y_train)
y_cross_lsvr = cross_lsvr.predict(ft_test)

cross_lsvr.score(ft_test, y_test)

0.7864102082971558

Interestingly enough, the linear model proved better than the linear SVR. Perhaps with a kernelized polynomial or rbf SVR, the results would be far better, but it isn't feasible to wait the requisite length of time.

The natural successor is the Random Forest Regressor. Since it uses randomized decision trees, it is substantially quicker than the Support Vector Machine, without sacrificing much accuracy.

In [22]:
from sklearn.ensemble import RandomForestRegressor as RFR

rfr = RFR()
rfr.fit(ft, y)
y_rfr = rfr.predict(ft)

rfr.score(ft, y)

0.98349664000295189

In [23]:
ft_train, ft_test, y_train, y_test = train_test_split(ft, y, test_size = .33, random_state=5)

cross_rfr = RFR()
cross_rfr.fit(ft_train, y_train)
y_cross_rfr = cross_rfr.predict(ft_test)

cross_rfr.score(ft_test, y_test)

0.89789405721214388

Wow! The R^2 value of .90 with the train-test split is a huge improvement over our preliminary approaches. Most importantly, this took substantially less time to run than most of the other models. 

# Further Investigations

There are a lot of parameters involved with the RFR, so a gridsearch on those could ostensibly provide a boost in speed and accuracy. There is also room to experiment with different features, including different elements of the sequences but especially statistics specific to time series. Lastly, ensembling methods could be used to combine multiple predictions.

Note that this project shifted away from pure accuracy in favor of using R^2 to test models. It would be interesting to see how the tested methods square up when binary prediction is required, and no points are awarded for being close enough.

# Applications

The Random Forest Regressor balanced accuracy and speed - making it a prime candidate for industry. While the kernelized SVR could have given an accuracy boost, its slowness precludes it from application.

These regressors can be used in any setting that uses sequential or time series data - including the stock market, weather patterns, migration behaviors, financial stability, and much more. Thus, these methods can prove integral to informed decision-making. Although in these notebooks the methods are aimed at the mathematical nature of integer sequence learning, they can easily be applied to the aforementioned practical scenarios. Remember that since the models were tested by their R^2 values, being close enough was valued over pure accuracy - a sacrifice that lends this research better to real-life problems.

In those situations, the balance between accuracy and speed is crucial - nobody will pay for a tool that takes forever to run, no matter how accurate, and a model that runs quickly but has no accuracy is about as useful as a Magic 8 Ball. Depending on the setting, the machine learning methods should be tuned and gridsearched for optimal parameters, but the results point to the Random Forest Regressor as a good base approach. Once the model is established, it makes sense to calculate confidence in the generated result - so decisions can be made probabilistically. If the model predicted a 5% increase in a stock's value, but with 95% confidence that the value would increase, you would most likely buy. This is the true power of applying machine learning to raw data. 

In summary, based on the research conducted, I recommend using Random Forest Regression in conjunction with cross-validation and grid search over tested alternatives to create a fast and accurate model for predicting various time series data in science or industrial settings. 

